In [18]:
%pip install groq

Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
os.environ['GROQ_API_KEY'] = "Sua chave aqui"

In [20]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)


In [21]:
class Agent:
    def __init__(self, client,  system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        resultado = self.execute()
        self.messages.append({"role": "assistant", "content": resultado})
        return resultado
    
    def execute(self):
        completion = client.chat.completions.create(
            messages=self.messages,
            model="llama-3.3-70b-versatile",
        )
        return completion.choices[0].message.content

Da mesma forma como foi mostrado na aula, aqui é criada a classe Agent que segue uma estrutura de inicialização pré-determinada pelo construtor, que é chamado de __init__. O primeiro método, chamado de __call__, é responsável pela chamada do objeto sempre que o ciclo se repetir, e nele, caso haja uma mensagem de sistema, a mensagem é adicionada na lista de mensagens e o mesmo chama o método "execute". Por último, o método "execute" é o responsável por enviar a requisição para o modelo e retornar a resposta recebida do modelo.

In [22]:
def calcular(operacao):
    return eval(operacao)

def buscar_preco_tenis(tenis):
    if tenis == "adidas":
        return 1099
    if tenis == "mizuno":
        return 999
    if tenis == "asics":
        return 2199
    if tenis == "nike":
        return 1199
    if tenis == "new balance":
        return 899
    if tenis == "fila":
        return 699
    if tenis == "olympikus":
        return 599
    return None

Aqui está a implementação das ferramentas disponíveis ao modelo. É fornecida uma pequena lista de produtos e uma função que permite fazer calculos com esses produtos.

In [23]:
system_prompt = """
Você funciona em um loop de Thought, Ação, PAUSA e Observação.
Ao final do loop, você deve produzir uma Answer.

Use Thought para descrever seu raciocínio sobre a pergunta recebida.
Use Ação para executar uma das ações disponíveis e depois retorne PAUSA.
Observação será o resultadoado da execução da ação.

Suas ações disponíveis são:

buscar_preco_tenis:
Exemplo: buscar_preco_tenis: adidas
Retorna o preço original do produto.

calcular:
Exemplo: calcular: (3 * 500) + (2 * 400)
Executa um cálculo matemático usando Python e retorna o resultadoado.
Use ponto para números decimais se necessário.

- Para aumentos percentuais, use a fórmula:
  valor_final = valor_original * (1 + percentual/100)
- Para descontos percentuais, use:
  valor_final = valor_original * (1 - percentual/100)

Exemplo de sessão:

Pergunta: Se eu comprar 2 tenis adidas e revendê-los com 10%% de aumento, qual será meu lucro?

Thought: Preciso consultar o preço do tenis adidas.
Ação: buscar_preco: adidas
PAUSA

Você será chamado novamente com:

Observação: 1099

Thought: Calcular o total da compra.
Ação: calcular: 2 * 1099
PAUSA

Você será chamado novamente com:

Observação: 2198

Thought: Calcular o valor de revenda com aumento de 10%.
Ação: calcular: 2198 * 1.10
PAUSA

Você será chamado novamente com:

Observação: 2417,8

Thought: Calcular o lucro.
Ação: calcular: 2417,8 - 2198
PAUSA

Você será chamado novamente com:

Observação: 219,8

Se você tiver a resposta final, produza:

Answer: O lucro total será de 219,8 reais.

Agora é sua vez:
""".strip()


Aqui é apenas o prompt que exemplifica a atuação do agente. Foi usado como base o prompt mostrado na aula, o mesmo foi apenas traduzido e modificado para se adequar ao objetivo do agente.

In [24]:
import re

def agent_loop(iteracoes_maximas, system, requisicao):
    agent = Agent(client, system_prompt)
    ferramentas = ['calcular', 'buscar_preco_tenis']
    proximo_promp = requisicao
    i = 0
    while i < iteracoes_maximas:
        i += 1
        resultado = agent(proximo_promp)
        print(resultado)

        if "PAUSA" in resultado and "Ação" in resultado:
            acao = re.findall(r"Ação: ([a-z_]+): (.+)", resultado, re.IGNORECASE)
            ferramenta_escolhida = acao[0][0]
            arg = acao[0][1]

            if ferramenta_escolhida in ferramentas:
                resultado_ferramenta = eval(f"{ferramenta_escolhida}('{arg}')")
                proximo_promp = f"Observação: {resultado_ferramenta}"
            
            else:
                proximo_promp = "Observação: Tool not found"

            print(proximo_promp)
            continue

        if "Answer" in resultado:
            break


Por fim, aqui é onde o loop em que o agente vai trabalhar é implementado. A cada iteração é enviado um novo prompt usando o método call do Agent, o que possibilita a analise das respostas fornecidas até chegar em um resultado final.

In [25]:
agent_loop(10, system_prompt, "Se eu comprar 2 tenis adidas e um da nike e revendê-los com 15% de aumento, qual será meu lucro?")

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}